# Human Detection Project

In [1]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
from sklearn.decomposition import PCA
from skimage.transform import pyramid_gaussian

# Lectura de imágenes

In [2]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

### Fotos Tain 

In [145]:
def duplicaTrani(fotos):
    indx = len(fotos)
    for i in range(len(fotos)):
        fotos[indx+i] = cv2.flip(fotos[i], 1)
        
    return fotos

In [146]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain0 = leerFotos(path)
fotosPosTrain0 = duplicaTrani(fotosPosTrain0)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

### Fotos Test

In [134]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotosTest0 = leerFotos(path)

### Recorte Train Neg

In [99]:
def generaImagenesRecortadas(winSize,dataNeg):    
    numeroCortesxPh = 20      # Numero de recortadas aleatorias por foto
    rowsFotoPersona = winSize[0]
    colsFotoPersona = winSize[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

In [100]:
winSize = fotosPosTrain0[0].shape
fotosNegTrain0 = generaImagenesRecortadas(winSize,fotosNegTrainSinTratar)

# Tratamiento de la imagenes Train

### Ampliazcion Rango Dinámico && Suavizado

In [92]:
def ampRangoSet(fotos):
    fotSal = {}
    for i in range(len(fotos)):
        r1 = np.min(fotos[i])
        r2 = np.max(fotos[i])
        fotSal[i] = np.float32(fotos[i])
        fotSal[i] = 255*(fotSal[i]-r1)/(r2-r1)
        fotSal[i] = np.uint8(fotSal[i])
        
    return fotSal

In [157]:
def trataFotos(fotos,kernelSize = 3):
    fotSal = {}
    ## Ampliación del rango
    fotSal = ampRangoSet(fotos)

    ## Suavizado
    kernel = np.ones((kernelSize,kernelSize))/(kernelSize**2)
    
    for i in range(len(fotSal)):
        fotSal[i] = cv2.filter2D(fotSal[i],-1,kernel)
        
    return fotSal

In [158]:
dimFiltro = 5
fotosNegTrain = trataFotos(fotosNegTrain0,dimFiltro)
fotosPosTrain = trataFotos(fotosPosTrain0,dimFiltro)
fotosTest = trataFotos(fotosTest0,dimFiltro)

# Extracción de Caracterísitcas
    L1
       Normalization using L1-norm.
    L1-sqrt
       Normalization using L1-norm, followed by square root.
    L2
       Normalization using L2-norm.
    L2-Hys
       Normalization using L2-norm, followed by limiting the
       maximum values to 0.2 (`Hys` stands for `hysteresis`) and
       renormalization using L2-norm. (default)

In [7]:
orientations    = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
block_norm      = 'L2' 
visualize       = False
visualise       = None
transform_sqrt  = True
feature_vector  = True

In [8]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector))

    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i,:] = hog(imagenes[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [159]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

In [160]:
fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

In [161]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

print(fotosTrain.shape)

(5126, 3780)


# Entrenamiento del modelo

In [162]:
clf = svm.LinearSVC(C = 0.007)
clf.fit(fotosTrain,y.ravel())

LinearSVC(C=0.007, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [163]:
clf.score(fdPosTrain,yPos)

0.9857904085257548

In [164]:
clf.score(fdNegTrain,yNeg)

0.999

# Slide Window

In [15]:
def slidWindow(imagen,maskHeight = 134,maskWidth = 70,step = 10):
    arrayImagen = []
    
    for i in range(0,imagen.shape[0]-maskHeight,step):        
        for j in range(0,imagen.shape[1]-maskWidth,step):
            arrayImagen.append((i,j,imagen[i:i+maskHeight, j:j+maskWidth]))
    
    return arrayImagen

# Pyramid Gaussian

In [16]:
def estudiaFoto(im,winSize,clf,ratio = 2):#Ratio de reducción
    # List to store the detections
    hdetections = []
    
    for scaLev,imScaled in enumerate(pyramid_gaussian(im,downscale=ratio,multichannel=False)):
        if (imScaled.shape[0] >= winSize[0] and imScaled.shape[1] >= winSize[1]):
            for (x,y,imVentana) in slidWindow(imScaled):
                fdSc = hog(imVentana,orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
                pred = clf.predict(fdSc.reshape(1,-1))
                if (pred == 1):
                    hdetections.append((x,y,int(winSize[1]*(ratio**scaLev)),int(winSize[0]*(ratio**scaLev)) ))
                    
    return hdetections

# Prueba del Modelo

In [165]:
def muestraResultado(detections,im):
    color = (255)
    for (x, y,w, h) in detections:
            # Draw the detections
            cv2.rectangle(im, (y, x), (y+w, x+h), color, thickness=1)
    cv2.imshow("Raw Detections before NMS", im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [166]:
%%time
humanDetec = estudiaFoto(fotosTest[0],winSize,clf,ratio=2)

Wall time: 15 s


In [167]:
len(humanDetec)

22

In [ ]:
muestraResultado(humanDetec,fotosTest[0].copy())

#### Prueba solo Una imagen

In [21]:
%%time
listOk = []
partImagen = slidWindow(fotosTest[6])
for i in range(len(partImagen)):
    pruebaHog = hog(partImagen[i][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector);
    if (1 == clf.predict(pruebaHog.reshape(1,-1))):
        listOk.append(i);

Wall time: 7.89 s


In [22]:
print(len(partImagen))
print(len(listOk))
print(listOk)

2730
57
[113, 238, 503, 766, 836, 856, 863, 882, 883, 925, 926, 1014, 1020, 1139, 1152, 1160, 1220, 1221, 1281, 1351, 1445, 1557, 1558, 1571, 1581, 1585, 1620, 1627, 1628, 1629, 1631, 1632, 1651, 1667, 1701, 1721, 1726, 1771, 1808, 1841, 1856, 1857, 1908, 1938, 1939, 1944, 1978, 2081, 2120, 2151, 2190, 2197, 2221, 2285, 2291, 2293, 2617]


In [23]:
cv2.imshow('img ', fotosTest[6])
cv2.imshow('trozo test ', partImagen[503][2])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
 pruebaHog = hog(partImagen[2291][2],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector,multichannel);
clf.decision_function(pruebaHog.reshape(1,-1))